In [1]:
"""
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name="google/flan-t5-small"

model_tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
"""

'\nfrom transformers import AutoTokenizer, AutoModelForSeq2SeqLM\nimport torch\n\ndevice=torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel_name="google/flan-t5-small"\n\nmodel_tokenizer=AutoTokenizer.from_pretrained(model_name)\nmodel=AutoModelForSeq2SeqLM.from_pretrained(model_name)\n'

In [2]:
"""
from datasets import load_dataset

superglue=load_dataset('super_glue', 'boolq')
superglue
"""

"from datasets import load_dataset\n\nsuperglue=load_dataset('super_glue', 'boolq')\nsuperglue"

In [3]:
"""
train=superglue['train']
test=superglue['test']
validation=superglue['validation']
"""

"train=superglue['train']\ntest=superglue['test']\nvalidation=superglue['validation']"

In [4]:
"""
import pandas as pd
train_df=pd.DataFrame(train)
test_df=pd.DataFrame(test)
val_df=pd.DataFrame(validation)
"""

'import pandas as pd\ntrain_df=pd.DataFrame(train)\ntest_df=pd.DataFrame(test)\nval_df=pd.DataFrame(validation)\n'

In [5]:
"""
train_df.head()
"""

'train_df.head()\n'

In [6]:
"""test_df.head()"""

'test_df.head()'

In [7]:
"""val_df.head()"""

'val_df.head()'

In [8]:
"""from evaluate import load
super_glue_metric=load('super_glue', 'boolq')
predictions = [0, 1]
references = [0, 1]
results = super_glue_metric.compute(predictions=predictions, references=references)
print(results)"""

"from evaluate import load\nsuper_glue_metric=load('super_glue', 'boolq')\npredictions = [0, 1]\nreferences = [0, 1]\nresults = super_glue_metric.compute(predictions=predictions, references=references)\nprint(results)"

In [9]:
"""
def tokenize_function(examples):
    def tokenize_function(examples, tokenizer):
        return tokenizer(examples["question"], padding="max_length", truncation=True)

    def tokenize_datasets(tokenizer, dataset):
        return dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    tokenized_train = tokenize_datasets(model_tokenizer, train)
    tokenized_test = tokenize_datasets(model_tokenizer, test)
    tokenized_validation = tokenize_datasets(model_tokenizer, validation)
    return model_tokenizer(examples["question"], padding="max_length", truncation=True)

"""

'\ndef tokenize_function(examples):\n    def tokenize_function(examples, tokenizer):\n        return tokenizer(examples["question"], padding="max_length", truncation=True)\n\n    def tokenize_datasets(tokenizer, dataset):\n        return dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)\n\n    tokenized_train = tokenize_datasets(model_tokenizer, train)\n    tokenized_test = tokenize_datasets(model_tokenizer, test)\n    tokenized_validation = tokenize_datasets(model_tokenizer, validation)\n    return model_tokenizer(examples["question"], padding="max_length", truncation=True)\n\n'

In [10]:
"""def  tokenize_dataset(examples):
    return model_tokenizer(examples['question'],padding="max_length",truncation=True)
token_data=superglue.map(tokenize_dataset,batched=True)
token_data"""

'def  tokenize_dataset(examples):\n    return model_tokenizer(examples[\'question\'],padding="max_length",truncation=True)\ntoken_data=superglue.map(tokenize_dataset,batched=True)\ntoken_data'

In [11]:
"""def get_predictions(model=model, dataset=token_data, tokenizer=model_tokenizer):
    input_ids = dataset['test']['input_ids'].to(device)
    attention_mask = dataset['test']['attention_mask'].to(device)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    return tokenizer.decode

def evaluate(predictions, references):
    results = super_glue_metric.compute(predictions=predictions, references=references)
    return results"""


"def get_predictions(model=model, dataset=token_data, tokenizer=model_tokenizer):\n    input_ids = dataset['test']['input_ids'].to(device)\n    attention_mask = dataset['test']['attention_mask'].to(device)\n    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)\n    return tokenizer.decode\n\ndef evaluate(predictions, references):\n    results = super_glue_metric.compute(predictions=predictions, references=references)\n    return results"

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from evaluate import load
import torch


model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset('super_glue', 'boolq', trust_remote_code=True)

/home/deon/miniconda3/envs/rakuten_project/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Using the latest cached version of the module from /home/deon/.cache/huggingface/modules/datasets_modules/datasets/super_glue/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed (last modified on Tue Sep 24 13:39:56 2024) since it couldn't be found locally at super_glue, or remotely on the Hugging Face Hub.


In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3245
    })
})

In [14]:
def add_instruction(examples):

    examples['question'] = [q + " Reply 1 for true and 0 for false." for q in examples['question']]
    return examples

dataset = dataset.map(add_instruction, batched=True)

def tokenize_fn(examples):
    return tokenizer(examples['question'], examples['passage'], truncation=True, padding=True)

tokenized_dataset = dataset.map(tokenize_fn, batched=True)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_predictions(example):
    inputs = tokenizer(example['question'], example['passage'], return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [30]:
tokenized_dataset = tokenized_dataset['validation'].map(lambda x: {'prediction': get_predictions(x)})

def convert_predictions_to_label(prediction):
    return 1 if prediction.lower() in ['true','yes',"1"] else 0



accuracy_metric = load('accuracy')



KeyError: "Column validation not in the dataset. Current columns in the dataset: ['question', 'passage', 'idx', 'label', 'input_ids', 'attention_mask', 'prediction', 'pred_label']"

In [ ]:
def convert_to_int()

In [20]:
tokenized_dataset = tokenized_dataset.map(lambda x: {'pred_label': convert_predictions_to_label(x['prediction'])})



Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset['prediction']

['0',
 'yes',
 '0',
 '1',
 '0',
 'no',
 'Yes',
 'is there a word with q without u',
 '0',
 '0',
 '0',
 'No',
 '0',
 '1',
 "['Fantastic Beasts and Where to Find Them']",
 'yes',
 '1',
 '1',
 'The Resident is an American medical drama television series aired by Fox Broadcasting Company',
 '1',
 'yes',
 'yes --',
 '0',
 '1',
 'Yes',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'yes',
 '0',
 '0',
 'yes',
 '1',
 '0',
 '1',
 '1',
 'yes',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 "Elena's heart",
 '0',
 '0',
 'yes',
 '1',
 '1',
 'No.',
 '0',
 '1',
 '0',
 '1',
 'yes',
 '0',
 '0',
 'yes',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 'Yes',
 'no',
 '0',
 'yes',
 '1',
 'yes',
 'German Shepherd Dog is a relatively new breed of dog, with their origin dating to 1899',
 '0',
 '0',
 'yes',
 '0',
 '1',
 'yes',
 '0',
 '1',
 '0',
 'No',
 '0',
 '0',
 '1 for true and 0 for false',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 'Gambling in Australia is legal i

In [53]:
predictions = tokenized_dataset['pred_label']
references = tokenized_dataset['label']

In [56]:
results = accuracy_metric.compute(predictions=predictions, references=references)
print("Accuracy:", results['accuracy'])

Accuracy: 0.3785932721712538


In [57]:
from evaluate import load
super_glue_metric=load('super_glue', 'boolq')

super_glue_metric.compute(predictions=predictions, references=references)

{'accuracy': 0.3785932721712538}

In [22]:
import pandas as pd
df_test=pd.DataFrame(dataset['test'])
df_test.head()

,question,passage,idx,label
0,is the first series 20 euro note still legal t...,20 euro note -- Until now there has been only ...,0,-1
1,do the champions league winners get automatic ...,2018–19 UEFA Champions League -- The final wil...,1,-1
2,can a bull snake kill a small dog Reply 1 for ...,Bullsnake -- Bullsnakes are very powerful cons...,2,-1
3,are all nba playoff games best of 7 Reply 1 fo...,NBA playoffs -- All rounds are best-of-seven s...,3,-1
4,can i use my train ticket on the tram in manch...,Manchester station group -- The Manchester sta...,4,-1


In [23]:
df_test['label'].value_counts()

label
-1    3245
Name: count, dtype: int64

In [26]:
df_val=pd.DataFrame(dataset['validation'])
df_val.head()

,question,passage,idx,label
0,does ethanol take more energy make that produc...,Ethanol fuel -- All biomass goes through at le...,0,0
1,is house tax and property tax are same Reply 1...,Property tax -- Property tax or 'house tax' is...,1,1
2,is pain experienced in a missing body part or ...,Phantom pain -- Phantom pain sensations are de...,2,1
3,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts -- ...,3,1
4,is there a difference between hydroxyzine hcl ...,Hydroxyzine -- Hydroxyzine preparations requir...,4,1


In [27]:
df_val['label'].value_counts()

label
1    2033
0    1237
Name: count, dtype: int64

In [28]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3270 entries, 0 to 3269
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  3270 non-null   object
 1   passage   3270 non-null   object
 2   idx       3270 non-null   int64 
 3   label     3270 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 102.3+ KB
